In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
data = pd.read_csv("/home/eiji/project/g/preprocessed_data/Fridge_LE.csv")
data

,hour,minute,second,fridge_temperature,temp_condition_encoded,label
0,12,36,52,13.10,1,0
1,12,36,53,8.65,1,0
2,12,36,54,2.00,0,0
3,12,36,55,4.80,0,0
4,12,36,56,10.70,1,0
...,...,...,...,...,...,...
587071,12,41,17,4.00,0,0
587072,12,41,17,6.05,1,0
587073,12,41,17,12.80,1,0
587074,12,41,18,6.50,1,0


In [2]:
import tensorflow as tf
tf.config.set_visible_devices([], 'GPU')

2024-09-20 15:24:16.912748: I tensorflow/core/util/port.cc:113] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-09-20 15:24:22.182331: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX_VNNI FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-09-20 15:24:28.309419: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
2024-09-20 15:24:37.029920: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:998] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/t

In [3]:
X = data[['hour', 'minute', 'second', 'fridge_temperature', 'temp_condition_encoded']].values
y = data['label'].values

In [ ]:
import numpy as np

# パラメータ設定
window_size = 30
train_ratio = 0.7
val_ratio = 0.15
n_samples = len(X)

# ウィンドウデータの作成
X_windows = []
y_windows = []

for i in range(n_samples - window_size + 1):
    X_windows.append(X[i:i + window_size])
    y_windows.append(y[i + window_size - 1])

X_windows = np.array(X_windows)
y_windows = np.array(y_windows)

# データブロックの定義
block_size = 100  # ブロックサイズの設定
n_blocks = len(X_windows) // block_size

X_train, y_train, X_val, y_val, X_test, y_test = [], [], [], [], [], []

for b in range(n_blocks):
    start = b * block_size
    end = start + block_size
    X_block = X_windows[start:end]
    y_block = y_windows[start:end]

    # ブロック内のサンプリング
    train_end = int(block_size * train_ratio)
    val_end = train_end + int(block_size * val_ratio)

    X_train.extend(X_block[:train_end])
    y_train.extend(y_block[:train_end])
    X_val.extend(X_block[train_end:val_end])
    y_val.extend(y_block[train_end:val_end])
    X_test.extend(X_block[val_end:])
    y_test.extend(y_block[val_end:])

# 配列に変換
X_train = np.array(X_train)
y_train = np.array(y_train)
X_val = np.array(X_val)
y_val = np.array(y_val)
X_test = np.array(X_test)
y_test = np.array(y_test)

# 結果のサイズを出力して確認
print("Training set size: ", X_train.shape)
print("Validation set size: ", X_val.shape)
print("Test set size: ", X_test.shape)


In [ ]:
window_size = 30
X_windows = []
y_windows = []

for i in range(len(X) - window_size + 1):
    X_windows.append(X[i:i + window_size])
    y_windows.append(y[i + window_size - 1])  # ラベルはウィンドウの最後のデータに対応させます

X_windows = np.array(X_windows)
y_windows = np.array(y_windows)


# トレーニングデータとテストデータに分割
X_train, X_temp, y_train, y_temp = train_test_split(X_windows, y_windows, test_size=0.3, random_state=42, stratify=y_windows)
# テストデータをさらに検証データとテストデータに分割
X_val, X_test, y_val, y_test = train_test_split(X_temp, y_temp, test_size=0.5, random_state=42, stratify=y_temp)

In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense

# LSTMモデルの構築
model = Sequential()
model.add(LSTM(50, input_shape=(X_train.shape[1], X_train.shape[2])))
model.add(Dense(1, activation='sigmoid'))

# モデルのコンパイル
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

# モデルの概要
model.summary()

In [ ]:
# モデルの訓練
# モデルの訓練
history = model.fit(X_train, y_train, epochs=10, batch_size=64, validation_data=(X_val, y_val), verbose=1, shuffle=False)


In [ ]:
# モデルの評価
loss, accuracy = model.evaluate(X_test, y_test, verbose=1)
print(f'Test Accuracy: {accuracy:.4f}')

In [ ]:
from sklearn.metrics import confusion_matrix, classification_report
import seaborn as sns
import matplotlib.pyplot as plt

# 予測の実行
y_pred = model.predict(X_test)
y_pred_classes = (y_pred > 0.5).astype(int)  # 閾値0.5でクラスに変換

# 混同行列の計算
conf_matrix = confusion_matrix(y_test, y_pred_classes)

# 混同行列の表示
plt.figure(figsize=(8, 6))
sns.heatmap(conf_matrix, annot=True, fmt='d', cmap='Blues')
plt.title('Confusion Matrix')
plt.xlabel('Predicted Label')
plt.ylabel('True Label')
plt.show()

# 精度、適合率、再現率、F1スコアのレポート
print(classification_report(y_test, y_pred_classes))


In [ ]:
import tensorflow as tf

# TensorFlowのバージョンを確認
print("TensorFlow version:", tf.__version__)

# 利用可能なGPUリストを表示
print("Available GPU devices:", tf.config.list_physical_devices('GPU'))
